In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd
import pickle
from pathlib import Path
from kuma_utils.utils import sigmoid
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score

from datasets import *
from transforms import *
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

In [4]:
test = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/sample_submission.csv')
valid = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v18v.csv')
public748 = pd.read_csv('input/public748.csv')

In [5]:
model_paths = [
    # Path('results/ds_09_val0'),
    Path('results/ds_09_val1'),
    # Path('results/aug_04'),
]


In [6]:
prediction_df = {}
oof_df = {}
for p in model_paths:
    if (p/'predictions_tta_wo_anomaly.npy').exists():
        prediction_df[p.stem] = sigmoid(np.load(p/'predictions_tta_wo_anomaly.npy')).reshape(-1)
    else:
        pred = sigmoid(np.load(p/'predictions_tta.npy'))
        oof = sigmoid(np.load(p/'outoffolds.npy'))
        print(pred.shape)
        if pred.shape[1] == 1:
            pred = pred.reshape(-1)
        elif pred.shape[0] == 5:
            pred = pred[0].reshape(-1)
        prediction_df[p.stem] = pred
        oof_df[p.stem] = oof.reshape(-1)
prediction_df = pd.DataFrame(prediction_df)
oof_df = pd.DataFrame(oof_df)
prediction_df['public748'] = public748['target'].values

(7975, 1)


In [8]:
valid = pd.concat([valid, oof_df], axis=1)

In [10]:
roc_auc_score(valid['target'], valid['ds_09_val1'])

0.7306230437939667

In [11]:
prediction_df

,ds_09_val1,public748
0,0.378731,0.077132
1,0.968598,0.848641
2,0.337151,0.106932
3,0.439851,0.256063
4,0.663693,0.507310
...,...,...
7970,0.999416,0.937471
7971,0.486552,0.088365
7972,0.262954,0.081671
7973,0.448427,0.112314


In [13]:
# test['target'] = prediction_df['aug_04'] * 0.2 + prediction_df['ds_05_aug2'] * 0.2 + prediction_df['public748'] * 0.6
test['target'] = prediction_df['ds_09'] * 0.33 + prediction_df['ds_13'] * 0.33 + prediction_df['public748'] * 0.33
# test['target'] = prediction_df['ds_09_mod0']

In [14]:
test

,id,target
0,00054c878,0.388141
1,0007285a3,0.916088
2,00076c5a6,0.284331
3,001349290,0.422014
4,001a52e92,0.606035
...,...,...
7970,ffbce04ef,0.969312
7971,ffc2d976b,0.327431
7972,ffc905909,0.234091
7973,ffe276f3e,0.290220


In [15]:
test.to_csv('results/blend_submission.csv', index=False)
# test.to_csv(Path('results/ds_09_mod0')/'submission.csv', index=False)